# Compare NLP Techniques: Build Model On doc2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

### Create doc2vec Vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets
tagged_docs_train = [gensim.models.doc2vec.TaggedDocument(v, [i])
                     for i, v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument(v, [i])
                    for i, v in enumerate(X_test['clean_text'])]

In [3]:
# What do these TaggedDocument objects look like?
tagged_docs_train[:10]

[TaggedDocument(words=&quot;[&#39;cool&#39;, &#39;text&#39;, &#39;head&#39;]&quot;, tags=[0]),
 TaggedDocument(words=&quot;[&#39;aiyo&#39;, &#39;u&#39;, &#39;always&#39;, &#39;c&#39;, &#39;ex&#39;, &#39;one&#39;, &#39;dunno&#39;, &#39;abt&#39;, &#39;mei&#39;, &#39;reply&#39;, &#39;first&#39;, &#39;time&#39;, &#39;u&#39;, &#39;reply&#39;, &#39;fast&#39;, &#39;lucky&#39;, &#39;workin&#39;, &#39;huh&#39;, &#39;got&#39;, &#39;bao&#39;, &#39;ur&#39;, &#39;sugardad&#39;, &#39;ahgee&#39;, &#39;&#39;]&quot;, tags=[1]),
 TaggedDocument(words=&quot;[&#39;ya&#39;, &#39;nice&#39;, &#39;ready&#39;, &#39;thursday&#39;]&quot;, tags=[2]),
 TaggedDocument(words=&quot;[&#39;think&#39;, &#39;still&#39;, &#39;car&#39;]&quot;, tags=[3]),
 TaggedDocument(words=&quot;[&#39;every&#39;, &#39;mondaynxt&#39;, &#39;week&#39;, &#39;vl&#39;, &#39;completing&#39;]&quot;, tags=[4]),
 TaggedDocument(words=&quot;[&#39;ok&#39;, &#39;knackered&#39;, &#39;came&#39;, &#39;home&#39;, &#39;went&#39;, &#39;sleep&#39;, &#39;go

In [4]:
# Train a basic doc2vec model
d2v_model = gensim.models.Doc2Vec(tagged_docs_train, vector_size=100, window=5, min_count=2)

In [5]:
# Infer the vectors to be used in training and testing
train_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]
test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]

### Fit RandomForestClassifier On Top Of Document Vectors

In [6]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.594 / Recall: 0.279 / Accuracy: 0.88
